In [1]:
import pandas as pd

import numpy as np
import matplotlib.pyplot as plt

In [2]:
dataset = pd.read_csv('Social_Network_Ads.csv')

In [3]:
dataset

,User ID,Gender,Age,EstimatedSalary,Purchased
0,15624510,Male,19,19000,0
1,15810944,Male,35,20000,0
2,15668575,Female,26,43000,0
3,15603246,Female,27,57000,0
4,15804002,Male,19,76000,0
...,...,...,...,...,...
395,15691863,Female,46,41000,1
396,15706071,Male,51,23000,1
397,15654296,Female,50,20000,1
398,15755018,Male,36,33000,0


In [4]:
dataset = pd.get_dummies(dataset,drop_first = True)

In [5]:
dataset

,User ID,Age,EstimatedSalary,Purchased,Gender_Male
0,15624510,19,19000,0,1
1,15810944,35,20000,0,1
2,15668575,26,43000,0,0
3,15603246,27,57000,0,0
4,15804002,19,76000,0,1
...,...,...,...,...,...
395,15691863,46,41000,1,0
396,15706071,51,23000,1,1
397,15654296,50,20000,1,0
398,15755018,36,33000,0,1


In [6]:
dataset.columns

Index(['User ID', 'Age', 'EstimatedSalary', 'Purchased', 'Gender_Male'], dtype='object')

In [7]:
dataset = dataset.drop('User ID' , axis = 1)

In [8]:
dataset

,Age,EstimatedSalary,Purchased,Gender_Male
0,19,19000,0,1
1,35,20000,0,1
2,26,43000,0,0
3,27,57000,0,0
4,19,76000,0,1
...,...,...,...,...
395,46,41000,1,0
396,51,23000,1,1
397,50,20000,1,0
398,36,33000,0,1


In [9]:
dataset['Purchased'].value_counts()

0    257
1    143
Name: Purchased, dtype: int64

In [10]:
indep = dataset[['Age', 'EstimatedSalary', 'Gender_Male']]

dep = dataset['Purchased']

In [11]:
indep.shape

(400, 3)

In [12]:
indep

,Age,EstimatedSalary,Gender_Male
0,19,19000,1
1,35,20000,1
2,26,43000,0
3,27,57000,0
4,19,76000,1
...,...,...,...
395,46,41000,0
396,51,23000,1
397,50,20000,0
398,36,33000,1


In [13]:
dep

0      0
1      0
2      0
3      0
4      0
      ..
395    1
396    1
397    1
398    0
399    1
Name: Purchased, Length: 400, dtype: int64

In [14]:
from sklearn.model_selection import train_test_split

X_train,X_test,Y_train,Y_test = train_test_split(indep,dep, test_size = 0.30, random_state = 0)

In [15]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

In [16]:
from sklearn.svm import SVC

from sklearn.model_selection import GridSearchCV

param_grid = {'kernel' : ['rbf','linear','poly','sigmoid'],
              'gamma': ['auto', 'scale']
              }
grid = GridSearchCV(SVC(probability = True),param_grid,refit = 'True', verbose = 3,cv = 5,n_jobs = -1, scoring = 'f1_weighted')
#fiiting the model for grid
grid.fit(X_train,Y_train)



Fitting 5 folds for each of 8 candidates, totalling 40 fits


GridSearchCV(cv=5, estimator=SVC(probability=True), n_jobs=-1,
             param_grid={'gamma': ['auto', 'scale'],
                         'kernel': ['rbf', 'linear', 'poly', 'sigmoid']},
             refit='True', scoring='f1_weighted', verbose=3)

In [17]:
#print best parameter after tuning 
#print(grid.best_params_)

re = grid.cv_results_

grid_predictions = grid.predict(X_test)



from sklearn.metrics import confusion_matrix
cm = confusion_matrix (Y_test,grid_predictions)

#print classification report

from sklearn.metrics import classification_report

clf_report  = classification_report(Y_test,grid_predictions)

In [18]:
from sklearn.metrics import f1_score

f1_macro = f1_score(Y_test,grid_predictions,average = 'weighted')

print('The R_score vlaue for best parameter {}:'.format(grid.best_params_),f1_score)

The R_score vlaue for best parameter {'gamma': 'auto', 'kernel': 'rbf'}: <function f1_score at 0x000001E2DCFBA288>


In [19]:
print (cm)

[[74  5]
 [ 4 37]]


In [20]:
print (clf_report)

              precision    recall  f1-score   support

           0       0.95      0.94      0.94        79
           1       0.88      0.90      0.89        41

    accuracy                           0.93       120
   macro avg       0.91      0.92      0.92       120
weighted avg       0.93      0.93      0.93       120



In [21]:
from sklearn.metrics import roc_auc_score

roc_auc_score(Y_test,grid.predict_proba(X_test)[:,-1])

0.9691262735412165

In [22]:
table = pd.DataFrame.from_dict(re)

In [23]:
table

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_gamma,param_kernel,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,0.211878,0.011325,0.056569,0.009764,auto,rbf,"{'gamma': 'auto', 'kernel': 'rbf'}",0.874254,0.875644,0.823129,0.947015,0.964286,0.896866,0.051883,1
1,0.167103,0.066189,0.062566,0.022138,auto,linear,"{'gamma': 'auto', 'kernel': 'linear'}",0.812540,0.782971,0.644599,0.927778,0.890114,0.811600,0.098358,5
2,0.101542,0.015296,0.029582,0.024868,auto,poly,"{'gamma': 'auto', 'kernel': 'poly'}",0.791441,0.835985,0.658971,0.946153,0.826525,0.811815,0.092329,3
3,0.091748,0.020507,0.018389,0.007168,auto,sigmoid,"{'gamma': 'auto', 'kernel': 'sigmoid'}",0.750000,0.715601,0.601841,0.875548,0.800053,0.748608,0.090986,7
4,0.088149,0.011155,0.016791,0.004789,scale,rbf,"{'gamma': 'scale', 'kernel': 'rbf'}",0.874254,0.875644,0.823129,0.947015,0.964286,0.896866,0.051883,1
5,0.062763,0.011541,0.017191,0.010160,scale,linear,"{'gamma': 'scale', 'kernel': 'linear'}",0.812540,0.782971,0.644599,0.927778,0.890114,0.811600,0.098358,5
6,0.069759,0.015908,0.019989,0.008409,scale,poly,"{'gamma': 'scale', 'kernel': 'poly'}",0.791441,0.835985,0.658971,0.946153,0.826525,0.811815,0.092329,3
7,0.058567,0.006941,0.008195,0.001935,scale,sigmoid,"{'gamma': 'scale', 'kernel': 'sigmoid'}",0.726641,0.715601,0.601841,0.806465,0.800053,0.730120,0.074024,8
